# Load Packages

In [9]:
import os as os

import numpy as np
from scipy.linalg import cholesky, cho_solve
from scipy.special import softmax
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
from sklearn.neighbors import BallTree
import rff
import rff_time
import datetime
import random as rd 

# Functions

In [10]:
def get_training_data_one_time(filePath,obsVar = 'obs_pm2_5', baseModels = ['av_pred', 'gs_pred','cmaq_outs_pred', 'js_pred', 'caces_pred']):
	"""
	Reads the training file YEARLY_TRAINING_FILE and returns the training data along with model predictions on training data.
	"""
	# Read the training data file using pandas into a DataFrame.
    # training data for just one year
	training_data = pd.read_csv(filePath)
	print('Columns of the training file = {}'.format(training_data.columns.values))
	print('Shape of the training file = {}'.format(training_data.shape))
	# Shuffle the data row-wise
	training_data = training_data.sample(frac=1.0)
	# (n, 2) numpy array.
    # we do not have time because this is annual data
	X_train = training_data[['lat', 'lon']].values
	# (n, 1) numpy array.
	y_train = training_data[[obsVar]].values
 # assert checks whether a statement is true and returns an message if it is false
	assert X_train.shape[0] == y_train.shape[0], "The number of rows in X_train, {0} and y_train don't match {1}".format(X_train.shape[0], y_train.shape[0])
	# Models' predictions on training data, as a (n, 3).
    # we will need to adjust this if we have other predictors
	models_on_train = training_data[baseModels].values
	assert models_on_train.shape[0] == X_train.shape[0], "The number of rows in models_on_train, {0} and X_train don't match {1}".format(models_on_train.shape[0], X_train.shape[0])
	return X_train, y_train, models_on_train


In [11]:
def get_training_data_multi_time(filePath, obsVar = 'obs_pm2_5', baseModels = ['av_pred', 'gs_pred','cmaq_outs_pred', 'js_pred', 'caces_pred']):
	"""
	Reads the training file TRAINING_FILE and returns the training data along with model predictions on training data.
	"""
	# Read the training data file using pandas into a DataFrame.
	dtype_dict = {'year':str, 'month':str, 'day':str} # Read these as string, and others as default
	training_data = pd.read_csv(filePath, dtype = dtype_dict)
	print('Columns of the training file = {}'.format(training_data.columns.values))
	print('Shape of the training file = {}'.format(training_data.shape))
	# Throw negative or very large outlier values
    # data curation should take place elsewhere
	training_data = training_data[training_data['obs_pm25'] > 0]
	training_data = training_data[training_data['obs_pm25'] < 40]
	# Throw non-conterminous states
	training_data = training_data[training_data['lat'] < 49.5]
	training_data = training_data[training_data['lat'] > 24.5]
	training_data = training_data[training_data['lon'] < -66.5]
	training_data = training_data[training_data['lon'] > -125]
	# Shuffle the data row-wise
	training_data = training_data.sample(frac=1.0)
	# (n, 3) numpy array
	X_train = training_data[['lat', 'lon']].values
	dates = training_data[['year', 'month', 'day']].values
	dates_combined = []
	n = dates.shape[0]
	for i in range(n):
		dates_combined.append('{0}-{1}-{2}'.format(dates[i,0], dates[i,1], dates[i,2]))
	dates_combined = np.array(dates_combined)
	start_date = sorted(dates_combined)[0]
	#date_start = np.datetime64(start_date)
	date_start = np.datetime64('2010-01-01')
 # dates_delta is the julian day
	dates_delta = np.zeros(shape=(n,))
	for i in range(n):
		dates_delta[i] = np.datetime64(dates_combined[i]) - date_start
		dates_delta[i] = dates_delta[i] #% 365 + 10 * np.floor(dates_delta[i] / 365)
	X_train = np.column_stack((X_train, dates_delta))
	# (n, 1) numpy array.
	y_train = training_data[[obsVar]].values
	assert X_train.shape[0] == y_train.shape[0], "The number of rows in X_train, {0} and y_train don't match {1}".format(X_train.shape[0], y_train.shape[0])
	# Models' predictions on training data, as a (n, L) for L=6.
	models_on_train = training_data[baseModels].values
	assert models_on_train.shape[0] == X_train.shape[0], "The number of rows in models_on_train, {0} and X_train don't match {1}".format(models_on_train.shape[0], X_train.shape[0])
	np.save(RESULTS_DIR + '/X_train.npy', X_train) # Read as X_train = np.load(RESULTS_DIR + '/X_train.npy')
	np.save(RESULTS_DIR + '/y_train.npy', y_train)
	np.save(RESULTS_DIR + '/models_on_train.npy', models_on_train)
	return X_train, y_train, models_on_train, start_date

In [12]:
def get_prediction_data(filePath, baseModels = ['av_pred', 'gs_pred','cmaq_outs_pred', 'js_pred', 'caces_pred'], sampleFrac = 1, timeStamp = 0, start_date = '2010-01-01', pred_date='2010-01-01'):
	"""
	Reads the prediction file YEARLY_PREDICTION_FILE and returns
	locations : (m, 2) numpy array
	predictions : (m, L) numpy array
	"""
	pred_data = pd.read_csv(filePath)
	if timeStamp != 0:
		pred_data = pred_data[pred_data['time'] == timeStamp]
	print('Columns of the predictions file = {}'.format(pred_data.columns.values))
	print('Shape of the predictions file = {}'.format(pred_data.shape))
	print('After sampling...')
	pred_data = pred_data.sample(frac=sampleFrac)
	print('Shape of the predictions file = {}'.format(pred_data.shape))
	locations = pred_data[['lat', 'lon']].values
	date_start = np.datetime64(start_date)
	date_pred = np.datetime64(pred_date)
	date_pred_num = date_pred - date_start
	dates_delta = [date_pred_num] * locations.shape[0]
	dates_delta = pd.to_timedelta(dates_delta).days
	locations = np.column_stack((locations, dates_delta))
	predictions = pred_data[baseModels].values
	return locations, predictions

In [13]:
def compute_hessian(Z_train, models_on_train, C, model_avg, error, sigmasq, lmbda=0.1):
	"""
	Computes the D(L+1) x D(L+1) Hessian.
	For L = 3, it looks like as follows:
	-----------------
	|w_0|0,1|0,2|0,3|
	-----------------
	|1,0|w_1|1,2|1,3|
	-----------------
	|2,0|2,1|w_2|2,3|
	-----------------
	|3,0|3,1|3,2|w_3|
	-----------------
	w_l denotes the DxD block nabla_{w_l}^2
	l,j denotes the DxD block nabla_{w_l} nabla_{w_j}

	# Inputs
	Z_train : (n, D) numpy array containing n projected data points
	models_on_train : (n, L) numpy array containing the model predictions on the training inputs
	C : (n, L) numpy array containing probabilitic weights of each model. Each row sums to 1
	model_avg = (n, 1) numpy array containing the models' average prediction obained from models_on_train and C
	error : (n, 1) numpy array containing the difference between the true values and ensemble prediction
	sigmasq : variance of the noise
	lmbda : the prior on the weights is N(0, 1/lmbda I_D)

	# Output
	"""
	n, D = Z_train.shape
	L = models_on_train.shape[1]
	# The Hessian initialized to 0.
	H = np.zeros(shape=(D*(L+1), D*(L+1)))
	# nabla_{w_0}^2 block
	nabla_0_0 = - (1/sigmasq) * np.matmul(Z_train.T, Z_train) - lmbda * np.identity(D)
	# Put nabla_0_0 in the top DxD block of H
	H[:D, :D]= nabla_0_0
	# (n, L) matrix storing the difference between a model prediction and models' average
	models_diff = models_on_train - model_avg
	# nabla_{w_l} nabla_{w_0} blocks
	for l in range(L):
		nabla_l_0 = - (1/sigmasq) * np.matmul(Z_train.T, np.multiply(Z_train, np.multiply(C[:, l, None], models_diff[:, l, None])))
		# Put nabla_l_0 in the corresponding positions
		H[(l+1)*D:(l+2)*D, :D] = nabla_l_0
		H[:D, (l+1)*D:(l+2)*D] = nabla_l_0
	# nabla_{w_l}^2 blocks
	for l in range(L):
		factor = np.multiply(C[:, l, None], np.multiply(models_diff[:, l, None], np.multiply(C[:, l, None], models_diff[:, l, None]) + 2 * np.multiply(C[:, l, None], error) - error))
		nabla_l_l = -lmbda * np.identity(D) - (1/sigmasq) * np.matmul(Z_train.T, np.multiply(Z_train, factor))
		# Put nabla_l_l in H
		H[(l+1)*D:(l+2)*D, (l+1)*D:(l+2)*D] = nabla_l_l
	# nabla_{w_j} nabla_{w_l} blocks
	for j in range(L):
		for l in range(j+1, L):
			factor = np.multiply(C[:, j, None], np.multiply(C[:, l, None], np.multiply(models_diff[:, j, None], models_diff[:, l, None]) + np.multiply(error, models_diff[:, l, None]) + np.multiply(error, models_diff[:, j, None])))
			nabla_j_l = - (1/sigmasq) * np.matmul(Z_train.T, np.multiply(Z_train, factor))
			# Put nabla_j_l in H
			H[(j+1)*D:(j+2)*D, (l+1)*D:(l+2)*D] = nabla_j_l
			H[(l+1)*D:(l+2)*D, (j+1)*D:(j+2)*D] = nabla_j_l
	return H


In [21]:
def fit_bne(X_train, y_train, models_on_train, D=500, h=20.0, ht=20.0, sigmasq=-1.0, lmbda=1.0, lmbda0=1.0):
	"""
	Trains the RFF approximation of BNE using gradient ascent.

	# Inputs
	X_train : (n, d) numpy array containing the "raw" data inputs
	y_train : (n, 1) numpy array containing the monitor observations
	models_on_train : (n, L) numpy array containing the model predictions on the training inputs
	D : Number of RFF features
	h : length scale for the Gaussian kernel exp(-|x-y|^2 / (2 * h)). The corresponding spectral measure is N(0, 1/h I_d)
	ht : length scale for time
	sigmasq : variance of the noise
	lmbda : the prior on the weights is N(0, 1/lmbda I_D)
	"""
	n, d = X_train.shape
	L = models_on_train.shape[1]
    # change this line if doing spatial-only model
	np.random.seed(0)
	rr = rff_time.RFF_TIME(X_train, D=D, h=h, ht=ht)
	# SNR of 8
	if sigmasq < 0:
		sigmasq = np.var(y_train) / 8
	# (n, D) numpy array containing the "projected" data inputs
	Z_train = rr.get_Z(X_train)
	# Initialize the weights
	np.random.seed(0)
	w0 = np.random.normal(size=(D, 1))
	np.random.seed(0)
	W = np.random.normal(size=(D, L))
	# Hyperparameters for the gradient ascent
	MAX_ITER = 20
	MAX_STEP_SIZE = 0.1
	BATCH_SIZE = 5000
	SPLITS = int(np.ceil(n / BATCH_SIZE))
	LL = []
	for it in range(1, MAX_ITER+1):
		step_size = MAX_STEP_SIZE / np.sqrt(it)
		for b in range(SPLITS):
			# Construct (BATCH_SIZE, D) matrix used for training
			Z = Z_train[b*BATCH_SIZE : min((b+1)*BATCH_SIZE, n)]
			# (BATCH_SIZE, 1)
			y = y_train[b*BATCH_SIZE : min((b+1)*BATCH_SIZE, n)]
			# (BATCH_SIZE, L)
			models = models_on_train[b*BATCH_SIZE : min((b+1)*BATCH_SIZE, n)]
			### Update W ###
			# Fourier GP approximation of shape (BATCH_SIZE, L)
			G = np.matmul(Z, W)
			# Each row sums to 1. (BATCH_SIZE, L) shape
			C = softmax(G, axis=1)
			# At each point get the model average. Shape (BATCH_SIZE, 1)
			model_avg = np.sum(np.multiply(C, models), axis=1, keepdims=True)
			# Bias term. Shape (BATCH_SIZE, 1)
			bias = np.matmul(Z, w0)
			# Error. (BATCH_SIZE, 1)
			error = y - model_avg - bias
			# Compute gradient
			gradient_W = (1/sigmasq) * np.matmul(Z.T, np.multiply(C, np.multiply(np.tile(error, L), models - model_avg))) - lmbda * W
			W = W + step_size * gradient_W
			### Update w0 ###
			# Fourier GP approximation of shape (BATCH_SIZE, L)
			G = np.matmul(Z, W)
			# Each row sums to 1. (BATCH_SIZE, L) shape
			C = softmax(G, axis=1)
			# At each point get the model average. Shape (BATCH_SIZE, 1)
			model_avg = np.sum(np.multiply(C, models), axis=1, keepdims=True)
			# Error. (BATCH_SIZE, 1)
			error = y - model_avg - bias
			# Compute gradient
			gradient_w0 = (1/sigmasq) * np.matmul(Z.T, error) - lmbda0 * w0
			w0 = w0 + step_size * gradient_w0
		# Compute log joint likelihood
		if it % 10 == 0:
			print('Iteration: {}'.format(it))
			G = np.matmul(Z_train, W)
			C = softmax(G, axis=1) # rescale the weights 
			model_avg = np.sum(np.multiply(C, models_on_train), axis=1, keepdims=True) # compute model average
			bias = np.matmul(Z_train, w0) # the key thing is that we only update the bias every 10 iterations
			error = y_train - model_avg - bias
			LL.append((-1/(2*sigmasq)) * np.sum(error**2) - (lmbda/2) * np.sum(W**2) - (lmbda0/2) * np.sum(w0**2))
	# Compute the covariance matrix of the Laplace approximation
	G = np.matmul(Z_train, W)
	C = softmax(G, axis=1)
	model_avg = np.sum(np.multiply(C, models_on_train), axis=1, keepdims=True)
	bias = np.matmul(Z_train, w0)
	error = y_train - model_avg - bias
	H = compute_hessian(Z_train, models_on_train, C, model_avg, error, sigmasq, lmbda)
	# Covariance matrix is the inverse of the negative of the Hessian evaluated at MAP
	COV = np.linalg.inv(-H)
	return W, w0, COV, rr, LL


In [15]:
def get_predictions(locations, predictions, W, w0, rr):
	"""
	Get BNE predictions at locations.

	# Inputs
	locations : (m, 3) numpy array # location in space and time
	predictions : (m, L) numpy array
	W : (D, L) numpy array
	w0 : (D, 1) numpy array
	rr : RFF object
	m : number of points to predict on
	# Returns
	"""
	# (m, D) shape
	Z_pred = rr.get_Z(locations)
	# GP Fourier approximation of shape (m, L)
	G = np.matmul(Z_pred, W)
	# Each row sums to 1. (m, L) shape
	C = softmax(G, axis=1)
	# At each location in locations, get model average. Shape (m, 1)
	model_avg = np.sum(np.multiply(C, predictions), axis=1, keepdims=True)
	# Bias term. Shape (m, 1)
	bias = np.matmul(Z_pred, w0)
	bne_pred = model_avg + bias
    # C is the properly-sclaed weighted, model_avg is the core ensemble component 
    # bias is the systematic bias term, and bne_pred is the actual predictions
	return C, model_avg, bias, bne_pred

In [16]:
def get_uncertainty(locations, predictions, W_map, w0_map, COV, rr, mc_samples=500):
	m, L = predictions.shape
	D = W_map.shape[0]
	W_samples = np.empty(shape=(mc_samples, m, L))
	model_avg_samples = np.empty(shape=(mc_samples, m))
	w0_samples = np.empty(shape=(mc_samples, m))
	bne_pred_samples = np.empty(shape=(mc_samples, m))
	# (mc_samples, D*(L+1))
	np.random.seed(0)
	W_w0_samples = np.random.multivariate_normal(np.reshape(np.hstack((W_map, w0_map)), newshape=(D*(L+1), )), COV, size=mc_samples)
	for s in range(mc_samples):
		if s % 50 == 0:
			print(s)     
		W = np.reshape(W_w0_samples[s, :D*L], newshape=(D, L))
		w0 = np.reshape(W_w0_samples[s, D*L:], newshape=(D, 1))
		W_samp, model_avg_samp, w0_samp, bne_pred_samp = get_predictions(locations, predictions, W, w0, rr)  
		for l in range(L):
			W_samples[s, :, l] = W_samp[:, l] 
		model_avg_samples[s, :] = model_avg_samp[:, 0] 
		w0_samples[s, :] = w0_samp[:, 0] 
		bne_pred_samples[s, :] = bne_pred_samp[:, 0] 
	# calculate summary metrics 
	W_mean = np.mean(W_samples, axis=(0))
	W_std = np.sqrt(np.var(W_samples, axis=0))
	model_avg_mean = np.mean(model_avg_samples, axis=0)
	model_avg_std = np.sqrt(np.var(model_avg_samples, axis=0))
	w0_mean = np.mean(w0_samples, axis=0)
	w0_std = np.sqrt(np.var(w0_samples, axis=0))
	bne_preds_mean = np.mean(bne_pred_samples, axis=0)
	bne_preds_std = np.sqrt(np.var(bne_pred_samples, axis=0))
	# combine into a nice dataframe
	bne_out = np.column_stack((model_avg_mean, model_avg_std, w0_mean, w0_std, bne_preds_mean, bne_preds_std))
	bne_out = np.column_stack((W_mean, W_std, bne_out))
	colNames = ['w_mean_av', 'w_mean_gs', 'w_mean_cm', 'w_mean_js', 'w_mean_cc', 'w_sd_av', 'w_sd_gs', 'w_sd_cm', 'w_sd_js', 'w_sd_cc', 'ens_mean', 'ens_sd','bias_mean', 'bias_sd', 'pred_mean', 'pred_sd']
	bne_out_df = pd.DataFrame(bne_out, columns = colNames)
#C, model_avg, bias, bne_pred

# save to results directory
	
	# Compute the mean also
	return bne_preds_mean

In [17]:
def perform_cv(X_train, y_train, models_on_train, D=500, h=10.0, ht=10.0, lmbda=1.0, splits=20):
	print(datetime.datetime.now())
	n, d = X_train.shape
	chunk = int(n/splits)
	mae = []
	rmse = []
	coverage = []
	for split in range(splits):
		print('Split: {0}  at {1}'.format(split, datetime.datetime.now()))
# we can just add filter statements, and a column for leaveOutfold
		flag = np.full((n,), True)
		flag[split*chunk : min((split+1)*chunk, n)] = False
		X_train1 = X_train[flag]
		X_train2 = X_train[np.invert(flag)]
		y_train1 = y_train[flag]
		y_train2 = y_train[np.invert(flag)]
		models_on_train1 = models_on_train[flag]
		models_on_train2 = models_on_train[np.invert(flag)]
		print('D = {}'.format(D))
		print('h = {}'.format(h))
		print('ht = {}'.format(ht))
		print('lmbda = {}'.format(lmbda))
		W, w0, COV, rr, LL = fit_bne(X_train1, y_train1, models_on_train1, D=D, h=h, ht=ht, sigmasq=-1.0, lmbda=lmbda)
		bne_pred, model_avg, bias = get_predictions(X_train2, models_on_train2, W, w0, rr)
		# Errors
		mae.append(np.sum(np.abs(y_train2 - bne_pred))/y_train2.shape[0])
		rmse.append(np.sqrt(np.sum((y_train2 - bne_pred)**2)/y_train2.shape[0]))
		# Coverage
		emp_std = get_uncertainty(X_train2, models_on_train2, W, w0, COV, rr, mc_samples=100)
		covered = 0
		for i in range(y_train2.shape[0]):
			if y_train2[i,0] >= bne_pred[i,0] - 2 * emp_std[i] and y_train2[i,0] <= bne_pred[i,0] + 2 * emp_std[i]:
				covered += 1
		coverage.append( covered / y_train2.shape[0])
	print(datetime.datetime.now())
	return mae, rmse, coverage


# Fit BNE

## Name Locations

In [18]:
# combined training dataset location
trainAnnualspt = 'training_datasets/training_avgscmjscc_all.csv'
trainAnnualsp = 'training_datasets/training_annual_2010_all.csv'
predAnnualsp = "prediction_datasets/predictions_annual_2010_all.csv"

trainDaily = 'training_datasets/dailyTrainingData_2010-2015.csv'
predDaily = "prediction_datasets/daily_pred_2010-01-01_small.csv"

RESULTS_DIR = "./bne_out"

baseModelSet = ['pred_av', 'pred_gs','pred_cmaq_outs', 'pred_js', 'pred_caces']

## Gather Data

In [19]:
X_train, y_train, models_on_train, start_date = get_training_data_multi_time(trainDaily, 'obs_pm25', baseModelSet)

locations, predictions = get_prediction_data(predDaily, baseModelSet,  1, 0, start_date)

Columns of the training file = ['ref_id' 'lat' 'lon' 'year' 'month' 'day' 'obs_pm25' 'pred_js'
 'cmaq_ins_pred' 'pred_cmaq_outs' 'pred_gs' 'pred_caces' 'pred_av']
Shape of the training file = (586777, 13)
Columns of the predictions file = ['js_ref_id' 'lat' 'lon' 'pred_js' 'cmaq_ins_pred' 'pred_cmaq_outs'
 'pred_gs' 'pred_caces' 'pred_av']
Shape of the predictions file = (56090, 9)
After sampling...
Shape of the predictions file = (56090, 9)


## Fit BNE

In [22]:
W_map, w0_map, COV, rr, LL = fit_bne(X_train, y_train, models_on_train, D=500, h=20.0, ht=20.0, sigmasq=-1.0, lmbda=1.0)

Iteration: 10
Iteration: 20


# Generate Predictions

In [23]:
C, model_avg, bias, bne_pred = get_predictions(locations, predictions, W_map, w0_map, rr)

In [24]:
bneout = get_uncertainty(locations, predictions, W_map, w0_map, COV, rr, mc_samples=500)

/var/folders/bb/_j5q9pcj2cd8q5sfhgxb4qv40000gn/T/ipykernel_29956/2731505044.py:10: RuntimeWarning: covariance is not positive-semidefinite.
  W_w0_samples = np.random.multivariate_normal(np.reshape(np.hstack((W_map, w0_map)), newshape=(D*(L+1), )), COV, size=mc_samples)


0
50
100
150
200
250
300
350
400
450


In [ ]:
bne_out.to_csv(RESULTS_DIR +'/bne_test.csv')